In [10]:
import numpy as np


def compute_pagerank(G, alpha=0.85, q=None, max_iter=100, tol=1.0e-6):
    """
    Compute the PageRank of each node in the graph G using the power method.

    :param G: 2D numpy array representing the adjacency matrix of the graph
    :param alpha: Damping factor for the random walk
    :param q: Probability distribution for random jumps (uniform by default)
    :param max_iter: Maximum number of iterations for the power method
    :param tol: Tolerance for convergence
    :return: 1D numpy array containing the PageRank of each node
    """
    n = G.shape[0]  # Number of nodes

    # If q is not provided, use a uniform distribution
    if q is None:
        q = np.ones(n) / n

    # Initialize PageRank vector
    p = np.ones(n) / n

    # Normalize rows of G to represent probabilities
    row_sums = G.sum(axis=1)
    G_normalized = G / row_sums[:, np.newaxis]

    for _ in range(max_iter):
        p_new = alpha * G_normalized.T.dot(p) + (1 - alpha) * q

        # Check convergence
        if np.linalg.norm(p_new - p, ord=1) <= tol:
            break

        p = p_new

    return p


def simulate_advanced_pagerank_scenarios():
    """
    Simulate different PageRank scenarios based on strategies like link addition/removal, link quality adjustment, and keyword adjustment.

    :return: Best strategy and its PageRank vector
    """
    # Example graphs for different strategies (simplified)
    strategy_graphs = {
        "add_links": np.array([[0, 1, 1, 0], [1, 0, 1, 1], [1, 1, 0, 1], [0, 1, 1, 0]], dtype=float),
        "remove_links": np.array([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]], dtype=float),
        "adjust_link_quality": np.array([[0, 2, 0, 0], [2, 0, 3, 0], [0, 3, 0, 4], [0, 0, 4, 0]], dtype=float),
        "adjust_keywords": np.array([[0, 1, 2, 0], [1, 0, 0, 3], [2, 0, 0, 1], [0, 3, 1, 0]], dtype=float)
    }

    # Compute PageRank for each strategy
    strategy_pageranks = {strategy: compute_pagerank(
        graph) for strategy, graph in strategy_graphs.items()}

    # Find the best strategy
    best_strategy = max(strategy_pageranks,
                        key=lambda s: np.max(strategy_pageranks[s]))

    return best_strategy, strategy_pageranks[best_strategy]


# Run the simulation
best_strategy, best_pagerank = simulate_advanced_pagerank_scenarios()
best_strategy, best_pagerank

('adjust_link_quality',
 array([0.13436854, 0.28490779, 0.36563146, 0.21509221]))

In [11]:
from scipy.optimize import linprog


def megiddo_method(weights, max_iter=100, tol=1e-6):
    """
    Apply Megiddo's method for a simplified fractional PageRank optimization problem.

    :param weights: Weights for each vertex representing its importance
    :param max_iter: Maximum number of iterations
    :param tol: Tolerance for convergence
    :return: Approximate solution to the fractional programming problem
    """
    n = len(weights)
    delta = 1.0  # Initial delta
    solution = None

    for _ in range(max_iter):
        # Linear constraints for the parameterized problem
        A = [weights - delta * np.ones(n), -np.ones(n)]
        b = [0, -1]

        # Linear objective function
        c = -np.ones(n)

        # Solve the parameterized linear problem
        res = linprog(c, A_ub=A, b_ub=b, bounds=(0, None))

        if res.success:
            delta = sum(weights * res.x) / sum(res.x)
            solution = res.x
        else:
            break

        # Check convergence
        if res.fun > -tol:
            break

    return solution, delta


# Example weights representing the importance of each vertex
weights = np.array([1.5, 2.0, 0.5, 1.0])

# Apply Megiddo's method
pagerank_solution, optimal_delta = megiddo_method(weights)
pagerank_solution, optimal_delta

(None, 1.0)

In [12]:
import networkx as nx
import numpy as np


def compute_pagerank(G, alpha=0.85):
    """
    Compute the PageRank of each node in the graph G using the power iteration method.
    """
    N = len(G)
    A = nx.to_numpy_array(G)
    A = A / A.sum(axis=0)
    v = np.random.rand(N, 1)
    v = v / v.sum()

    for _ in range(100):
        v = alpha * A @ v + (1 - alpha) / N

    return {n: v[i][0] for i, n in enumerate(G.nodes())}


def is_nash_equilibrium(G, alpha=0.85):
    """
    Check if the current state of graph G is a Nash Equilibrium.
    """
    original_pageranks = compute_pagerank(G, alpha)

    for node in G.nodes():
        for neighbor in list(G.neighbors(node)):
            # Remove an edge and compute new PageRank
            G.remove_edge(node, neighbor)
            new_pageranks = compute_pagerank(G, alpha)

            # If the PageRank of the node increases by removing an edge, it's not a NE
            if new_pageranks[node] > original_pageranks[node]:
                G.add_edge(node, neighbor)  # restore the edge
                return False

            G.add_edge(node, neighbor)  # restore the edge

    return True


# Create a simple undirected graph
G = nx.Graph()
# G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 1), (1, 3)])
G.add_edges_from([(1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 3), (2, 4), (3, 1), (3, 4), (4, 1), (4, 3), (5, 1)])
# Compute PageRank and check Nash Equilibrium
pagerank_values = compute_pagerank(G)
is_ne = is_nash_equilibrium(G)

pagerank_values, is_ne

/var/folders/q8/h2vz6sj57rj02qh11jbnrysh0000gn/T/ipykernel_14458/4232707505.py:11: RuntimeWarning: invalid value encountered in divide
  A = A / A.sum(axis=0)


({1: 0.28405589394482267,
  2: 0.20852740953063417,
  3: 0.20852740953063417,
  4: 0.20852740953063417,
  5: 0.09036187746327481},
 True)